In [1]:
models = ['GIN', 'GAT', 'GCNN', 'GatedGCNN']
# note, no twitter.
TUdataset_names = ['ENZYMES', 'PROTEINS', 'IMDB-BINARY', 'MUTAG', 'COLLAB']
prefix = "230820_test2"

In [3]:
import os
import json
import pandas as pd
import numpy as np

folder_path = '/home/penlu/code/bespoke-gnn4do/training_runs'
model_list = [os.path.join(folder_path,x) for x in os.listdir(folder_path) if x.startswith(prefix)]

df = pd.DataFrame(columns=TUdataset_names)

# load in params
for model_folder in model_list:
    try:
        with open(os.path.join(model_folder, 'params.txt'), 'r') as f:
            model_args = json.load(f)
        losses = np.load(os.path.join(model_folder, 'losses.npy'))
        df.at[model_args['model_type'], model_args['TUdataset_name']] = losses[-1]
    except:
        print(f'something is wrong w/ {model_folder}')
        

In [4]:
df

,ENZYMES,PROTEINS,IMDB-BINARY,MUTAG,COLLAB
GCNN,3181.824631,3733.874811,37041.56134,7.075259,3725483.14502
GAT,-14371.183533,-32165.913513,-4699.307541,-1356.819946,-407940.139954
GIN,-6623.810577,-13129.591034,34920.449707,-1306.177155,3412673.52124
GatedGCNN,-18645.928589,-40293.291687,-6026.650932,-2875.71637,9132687.035156
